In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import keras_tuner
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [4]:
image_height=224
image_width=224
batch_size=16

In [6]:
ds_train=tf.keras.preprocessing.image_dataset_from_directory(
    "../data/animal_data",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(image_height,image_width),
    shuffle=True,
    seed=12,
    validation_split=.1,
    subset="training",
)

Found 1944 files belonging to 15 classes.
Using 1750 files for training.


In [9]:
ds_test=tf.keras.preprocessing.image_dataset_from_directory(
    "../data/animal_data",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(image_height,image_width),
    shuffle=True,
    seed=12,
    validation_split=.1,
    subset="validation"
)

Found 1944 files belonging to 15 classes.
Using 194 files for validation.


In [10]:
def normalize_img(image, label):
    # Normalize image
    normalized_image = tf.cast(image, tf.float32) / 255.0
    return normalized_image, label

In [12]:
# Prepare dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.prefetch(AUTOTUNE)
ds_test=ds_test.prefetch(AUTOTUNE)

In [15]:
ds_test

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>